<a href="https://colab.research.google.com/github/ibacaraujo/to-tensorflow2/blob/master/save_and_restore_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Save and restore models

In [1]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except:
  pass

!pip install -q pyyaml h5py # Required to save models in HDF5 format

TensorFlow 2.x selected.


In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

import os

import tensorflow as tf
from tensorflow import keras

print(tf.version.VERSION)

2.0.0-rc1


## Dataset

In [3]:
(train_images, train_labels), (test_images, test_labels) = keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28*28) / 255.0
test_images = test_images[:1000].reshape(-1, 28*28) / 255.0

11493376/11490434 [==============================] - 0s 0us/step


## The model

In [4]:
# Define a simple sequential model
def create_model():
  model = keras.Sequential([
    keras.layers.Dense(512, activation='relu', input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10, activation='softmax')
  ])

  model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

  return model

# Create a basic model instance
model = create_model()

# Display the model's architecture
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


## Save checkpoints during training

Train without having to retrain it or pick-up training where you left off - in case the training process was interrupted.

In [6]:
checkpoint_path = 'training_1/cp.ckpt'
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                              save_weights_only=True,
                                              verbose=1)

# Train the model with the new callback
model.fit(train_images,
          train_labels,
          epochs=10,
          validation_data=(test_images, test_labels),
          callbacks=[cp_callback]) # Pass callback to training

Train on 1000 samples, validate on 1000 samples
Epoch 1/10
 928/1000 [==========================>...] - ETA: 0s - loss: 1.2073 - accuracy: 0.6552
Epoch 00001: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 2s 2ms/sample - loss: 1.1648 - accuracy: 0.6680 - val_loss: 0.7048 - val_accuracy: 0.7790
Epoch 2/10
 896/1000 [=========================>....] - ETA: 0s - loss: 0.4256 - accuracy: 0.8862
Epoch 00002: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 0s 247us/sample - loss: 0.4166 - accuracy: 0.8870 - val_loss: 0.5306 - val_accuracy: 0.8340
Epoch 3/10
 896/1000 [=========================>....] - ETA: 0s - loss: 0.2668 - accuracy: 0.9353
Epoch 00003: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 0s 237us/sample - loss: 0.2690 - accuracy: 0.9340 - val_loss: 0.4687 - val_accuracy: 0.8570
Epoch 4/10
 544/1000 [===============>..............] - ETA: 0s - loss: 0.2034 - accuracy: 0.9577
Epoch 0

In [7]:
!ls {checkpoint_dir} # checkpoint files that are updated at the end of each epoch

checkpoint		     cp.ckpt.data-00001-of-00002
cp.ckpt.data-00000-of-00002  cp.ckpt.index


When restoring model's weights, to use an untrained model with the same architecture.

In [9]:
# Create a basic model instance
model = create_model()

# Evaluate the model
loss, acc = model.evaluate(test_images, test_labels)
print('Untrained model, accuracy: {:5.2f}%'.format(100*acc))

1000/1 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

Let's load the weights from the checkpoint and re-evaluate.

In [11]:
# Load the weights
model.load_weights(checkpoint_path)

# Re-evaluate the model
loss, acc = model.evaluate(test_images, test_labels)
print('Restored model, accuracy: {:5.2f}%'.format(100*acc))

1000/1 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

## Checkpoint callback options that are really used

In [12]:
# Train a model and save uniquely named checkpoints once every five epochs

# Include the epoch in the file name
checkpoint_path = 'training_2/cp-{epoch:04d}.ckpt'
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights every 5 epochs
cp_callback = tf.keras.callbacks.ModelCheckpoint(
  filepath=checkpoint_path,
  verbose=1,
  save_weights_only=True,
  period=5
)

# Create a new model instance
model = create_model()

# Save the weights using the 'checkpoint_path' format
model.save_weights(checkpoint_path.format(epoch=0))

# Train the model with the new callback
model.fit(train_images,
          train_labels,
          epochs=50,
          callbacks=[cp_callback],
          validation_data=(test_images,test_labels),
          verbose=0)


Epoch 00005: saving model to training_2/cp-0005.ckpt

Epoch 00010: saving model to training_2/cp-0010.ckpt

Epoch 00015: saving model to training_2/cp-0015.ckpt

Epoch 00020: saving model to training_2/cp-0020.ckpt

Epoch 00025: saving model to training_2/cp-0025.ckpt

Epoch 00030: saving model to training_2/cp-0030.ckpt

Epoch 00035: saving model to training_2/cp-0035.ckpt

Epoch 00040: saving model to training_2/cp-0040.ckpt

Epoch 00045: saving model to training_2/cp-0045.ckpt

Epoch 00050: saving model to training_2/cp-0050.ckpt


In [13]:
!ls {checkpoint_dir}

checkpoint			  cp-0025.ckpt.data-00001-of-00002
cp-0000.ckpt.data-00000-of-00002  cp-0025.ckpt.index
cp-0000.ckpt.data-00001-of-00002  cp-0030.ckpt.data-00000-of-00002
cp-0000.ckpt.index		  cp-0030.ckpt.data-00001-of-00002
cp-0005.ckpt.data-00000-of-00002  cp-0030.ckpt.index
cp-0005.ckpt.data-00001-of-00002  cp-0035.ckpt.data-00000-of-00002
cp-0005.ckpt.index		  cp-0035.ckpt.data-00001-of-00002
cp-0010.ckpt.data-00000-of-00002  cp-0035.ckpt.index
cp-0010.ckpt.data-00001-of-00002  cp-0040.ckpt.data-00000-of-00002
cp-0010.ckpt.index		  cp-0040.ckpt.data-00001-of-00002
cp-0015.ckpt.data-00000-of-00002  cp-0040.ckpt.index
cp-0015.ckpt.data-00001-of-00002  cp-0045.ckpt.data-00000-of-00002
cp-0015.ckpt.index		  cp-0045.ckpt.data-00001-of-00002
cp-0020.ckpt.data-00000-of-00002  cp-0045.ckpt.index
cp-0020.ckpt.data-00001-of-00002  cp-0050.ckpt.data-00000-of-00002
cp-0020.ckpt.index		  cp-0050.ckpt.data-00001-of-00002
cp-0025.ckpt.data-00000-of-00002  cp-0050.ckpt.index


In [14]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2/cp-0050.ckpt'

In [21]:
# Reset the model and load the latest checkpoint
model = create_model()

model.load_weights(latest)

loss, acc = model.evaluate(test_images, test_labels)
print('Restored model, accuracy: {:5.2f}%'.format(100*acc))

1000/1 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

## Manually save weights

In [22]:
# Save the weights
model.save_weights('./checkpoints/my_checkpoint')

model = create_model()

model.load_weights('./checkpoints/my_checkpoint')

# Evaluate the model
loss, acc = model.evaluate(test_images, test_labels)
print('Restored model, accuracy: {:5.2f}%'.format(100*acc))

1000/1 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

## Save the entire model

In [23]:
model = create_model()
model.fit(train_images, train_labels, epochs=5)
model.save('my_model.h5')

Train on 1000 samples
Epoch 1/5
1000/1000 [==============================] - 0s 458us/sample - loss: 1.0974 - accuracy: 0.6960
Epoch 2/5
1000/1000 [==============================] - 0s 108us/sample - loss: 0.4093 - accuracy: 0.8840
Epoch 3/5
1000/1000 [==============================] - 0s 123us/sample - loss: 0.2866 - accuracy: 0.9320
Epoch 4/5
1000/1000 [==============================] - 0s 111us/sample - loss: 0.2086 - accuracy: 0.9480
Epoch 5/5
1000/1000 [==============================] - 0s 116us/sample - loss: 0.1498 - accuracy: 0.9660


In [24]:
# Now recreate the model, the exact same model, from that file
new_model = keras.models.load_model('my_model.h5')
new_model.summary()

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_18 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_9 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_19 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [25]:
# Check its accuracy
loss, acc = new_model.evaluate(test_images, test_labels)
print('Restored model, accuracy: {:5.2f}%'.format(100*acc))

1000/1 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================